# Assignment - Artificial Intelligence 3

## Assignment Background

Timely submission will result in your assignment being returned within 14 days of the submission deadline. Marked Jupyter Notebook manuscripts will be returned with comments from your marker. If you receive a pass or distinction, your marker will attach the model solutions for this assignment *via* the open pull request on your lesson's GitHub repository.

The aim of this assignment is to recreate the workflow of the image section of this lesson, using a different dataset. The dataset will be the PneumoniaMNIST dataset that was used in the previous lesson's assignment.

Given the time required to train and optimise a model, we have trained a CNN that can be found in the [data](./data/pneumonia_model.pth) folder. The architecture has been provided for you to load the model, which achieved an accuracy of 85% on the test set.

Your task is to investigate how the model is making correct predictions (true positives: images correctly identified as pneumonia) and why it is making incorrect predictions (false negatives: pneumonia cases the model missed).

As these are real world datasets the findings from the solutions may not be immediately obvious, as we are not specialists in pneumonia chest x-rays, so do not worry too much about interpreting the output itself.

## Assignment Questions:

## Question 1: 

**Task 1:**
- Download the PneumoniaMNIST dataset
- Load the test dataset into a DataLoader
- Initialise the `PneumoniaCNN` model and load the saved weights from `./data/pneumonia_model.pth`

The code for most of this has already been given to you.

In [ ]:
import torch
import torch.nn as nn

import warnings
warnings.filterwarnings("ignore") # ignore warnings

# Import the MNIST dataset
from torchvision import transforms

# import medmnist, the medical image dataset
import medmnist

import matplotlib.pyplot as plt
import numpy as np

# fix the numpy and torch seeds for reproducibility:
SEED = 999
np.random.seed(SEED)
torch.manual_seed(SEED);

In [ ]:
data_flag = 'pneumoniamnist'

info = medmnist.INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

real_labels = {0: "Normal", 1: "Pneumonia"}

print(f"The learning task is {task}")
print(f"The number of channels is {n_channels}")
print(f"The number of classes is {n_classes}")

download = True
BATCH_SIZE = 128

DataClass = getattr(medmnist, info['python_class'])

# preprocessing
data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5]) # normalise the data with standard values
])

# Test dataset
test_loader = torch.utils.data.DataLoader(
    DataClass(split='test', 
                transform=data_transform, 
                download=download, 
                root='data'),
    batch_size=2*BATCH_SIZE, shuffle=True)

In [ ]:
class PneumoniaCNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=2):
        super().__init__()
        
        # First convolutional block
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels, 32, kernel_size=3, padding=1),  # 28x28 -> 28x28
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # 28x28 -> 14x14
        )
        
        # Second convolutional block
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),  # 14x14 -> 14x14
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # 14x14 -> 7x7
        )
        
        # Third convolutional block
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),  # 7x7 -> 7x7
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # 7x7 -> 3x3
        )
        
        # Fully connected layers
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 3 * 3, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )
    
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.fc(x)
        return x

In [ ]:
if torch.cuda.is_available():
    # if there are multiple GPUs, choose the first one
    device = torch.device(f"cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print(f'Device name: {torch.cuda.get_device_name(0)}')

elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print(f"Using: {device} (Apple Silicon)")

else:
    print("No GPU detected! Falling back to CPU")
    device = torch.device("cpu")

# then define the model (architecture above) and move it to the device:
model = PneumoniaCNN()

# load the trained model that we saved in the last lesson, instead of having to train it again:
model.load_state_dict(torch.load('./data/pneumonia_model.pth', weights_only=True))
model = model.to(device)

**Task 2:**
- Select **3** images from the test set that are correctly predicted by the model (true positives: predicted pneumonia, label is pneumonia).
- Select **3** images from the test set that are incorrectly predicted by the model (false negatives: predicted normal, label is pneumonia).
- In two separate plots, display the images and their predicted labels.

Some help: the code to load all test labels and run predictions is provided below. To filter the images, collect the **index** of each sample where the prediction and true label match the required condition. For true positives both values will **equal 1**; for false negatives the prediction will be **0** and the label **1**.

In [ ]:
# Load all test images and labels into memory
test_images = []
test_labels = []
for images, labels in test_loader:
    test_images.append(images)
    test_labels.append(labels)
test_images = torch.cat(test_images, dim=0)
test_labels = torch.cat(test_labels, dim=0).squeeze()

# Run the model on all test images
model.eval()
with torch.no_grad():
    output = model(test_images.to(device))

_, preds = torch.max(output, 1)

In [ ]:
# True positives: model predicts pneumonia and the label is pneumonia
correct_indices = [
    i for i in range(len(preds))
    if preds[i] == 1 and test_labels[i] == 1
]

np.random.shuffle(correct_indices)
correct_indices = correct_indices[:3]

correct_indices

In [ ]:
# Plot the images in the batch, along with predicted and true labels
fig, axes = plt.subplots(1, 3, figsize=(9, 3))

for ax, idx in zip(axes, correct_indices):
    ax.imshow(test_images[idx].squeeze(), cmap='gray')
    ax.set_title(f"{real_labels[int(preds[idx].cpu())]}", color="green")
    ax.axis('off')

fig.suptitle('Correct Predictions (True Positives)', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# False negatives: model predicts normal but the label is pneumonia
false_neg_indices = [
    i for i in range(len(preds))
    if preds[i] == 0 and test_labels[i] == 1
]

np.random.shuffle(false_neg_indices)
false_neg_indices = false_neg_indices[:3]

false_neg_indices

In [ ]:
# Plot the images in the batch, along with predicted and true labels
fig, axes = plt.subplots(1, 3, figsize=(9, 3))

for ax, idx in zip(axes, false_neg_indices):
    ax.imshow(test_images[idx].squeeze(), cmap='gray')
    ax.set_title(f"{real_labels[int(preds[idx].cpu())]}", color="red")
    ax.axis('off')

fig.suptitle('Incorrect Predictions (False Negatives)', fontsize=14)
plt.tight_layout()
plt.show()

## Question 2

**Task 1:**

Generate an occlusion attribution heatmap for each image in each set by completing the following:

- Create a function called `plot_occlusion_grid` that runs the occlusion attribution and displays results in a **3 × 2** grid of subplots — the greyscale image on the left, the occlusion heatmap on the right
- Set the sliding window shape and strides for an image with **1** channel
- Transpose the attribution output before plotting
- Use **Captum's** visualisation function with `use_pyplot=False` for the heatmap

All the code is in the lesson notebook and only needs to be adapted for a single-channel image and wrapped in a function.

In [ ]:
from captum.attr import Occlusion
from captum.attr import FeatureAblation
from captum.attr import Lime
from captum.attr import visualization as viz
import cv2
from skimage.filters import threshold_otsu

In [ ]:
def plot_occlusion_grid(image_indices, images, labels, model, device, title=""):
    """
    Runs occlusion attribution on a set of images and plots results in a 2 x 3 grid.

    Args:
        image_indices : list of 6 image indices to analyse
        images        : full test image tensor (N, C, H, W)
        labels        : full test label tensor (N,)
        model         : trained PyTorch model
        device        : torch device
        title         : suptitle for the figure
    """
    occlusion = Occlusion(model)

    # PneumoniaMNIST is 1-channel, so the sliding window has 1 channel (not 3 as in the lesson)
    sliding_window_shapes = (1, 2, 2)  # (channels, height, width)
    strides = (1, 1, 1)               # (channel_stride, height_stride, width_stride)

    fig, axes = plt.subplots(3, 2, figsize=(8, 12))

    model.eval()
        
    for row, idx in enumerate(image_indices):

        img = images[idx].unsqueeze(0).to(device)
        label = int(labels[idx])

        out = model(img)
        _, pred_class = torch.max(out, 1)

        # Run occlusion attribution
        attributions = occlusion.attribute(
            img,
            strides=strides,
            target=pred_class,
            sliding_window_shapes=sliding_window_shapes
        )

        # Transpose from (C, H, W) to (H, W, C) for the viz module
        attr_data = np.transpose(attributions.squeeze(0).cpu().detach().numpy(), (1, 2, 0))
        plot_img  = img.squeeze().cpu().numpy()  # (H, W) — already greyscale

        pred_text = real_labels[pred_class.item()]
        true_text = real_labels[label]

        # Left: greyscale image
        axes[row, 0].imshow(plot_img, cmap='gray')
        axes[row, 0].set_title(f"Pred: {pred_text}  |  True: {true_text}")
        axes[row, 0].axis('off')

        # Right: LIME attribution heatmap
        viz.visualize_image_attr(
            attr_data,
            plot_img,
            method='heat_map',
            sign='all',
            plt_fig_axis=(fig, axes[row, 1]),
            use_pyplot=False,
            show_colorbar=True
        )
        axes[row, 1].set_title("Occlusion")


    fig.suptitle(title, fontsize=14)
    plt.tight_layout()
    plt.show()

**Task 2**

1. Call `plot_occlusion_grid` with the true positive images
2. Call `plot_occlusion_grid` with the false negative images

Are there any obvious differences between the two sets?

In [ ]:
# Occlusion heatmaps for correct predictions (true positives)
plot_occlusion_grid(
    correct_indices, test_images, test_labels, model, device,
    title="Occlusion Attributions — Correct Predictions (True Positives)"
)

In [ ]:
# Occlusion heatmaps for false negatives
plot_occlusion_grid(
    false_neg_indices, test_images, test_labels, model, device,
    title="Occlusion Attributions — False Negatives"
)

## Question 3

**Task 1:**

Create a function called `plot_lime_grid` that calculates and plots segment-level attributions using LIME:

- Use Otsu thresholding followed by connected components to segment the image
- Convert the segment labels into a Captum feature mask and pass it to the LIME attribution method
- Plot the greyscale image on the left and the LIME attribution heatmap on the right in a **3 × 2** grid

In [ ]:
def plot_lime_grid(image_indices, images, labels, model, device, title=""):
    """
    Runs Otsu segmentation + LIME on 3 images and plots in a 3 x 2 grid.
    Left column : original image in greyscale.
    Right column: LIME attribution heatmap.
    """

    lime = Lime(model)
    fig, axes = plt.subplots(3, 2, figsize=(8, 12))

    model.eval()
    
    for row, idx in enumerate(image_indices):
        img   = images[idx].unsqueeze(0).to(device)
        label = int(labels[idx])

        with torch.no_grad():
            out = model(img)
        _, pred_class = torch.max(out, 1)

        # Greyscale image — already 1-channel, just squeeze to (H, W)
        plot_img = img.squeeze().cpu().numpy()

        # Otsu thresholding → binary mask → connected components
        thresh = threshold_otsu(plot_img)
        binary = (plot_img > thresh).astype(np.uint8)
        _, labels_map = cv2.connectedComponents(binary)

        # Build Captum feature mask
        mask = torch.from_numpy(labels_map.astype(np.int64)).unsqueeze(0).unsqueeze(0).to(device)

        # LIME attribution
        pred_class_lime = model(img).argmax(dim=1)
        attributions = lime.attribute(img, target=pred_class_lime, feature_mask=mask)
        attr_data = np.transpose(attributions.squeeze(0).cpu().detach().numpy(), (1, 2, 0))

        pred_text = real_labels[pred_class.item()]
        true_text = real_labels[label]

        # Left: greyscale image
        axes[row, 0].imshow(plot_img, cmap='gray')
        axes[row, 0].set_title(f"Pred: {pred_text}  |  True: {true_text}")
        axes[row, 0].axis('off')

        # Right: LIME attribution heatmap
        viz.visualize_image_attr(
            attr_data,
            plot_img,
            method='heat_map',
            sign='all',
            plt_fig_axis=(fig, axes[row, 1]),
            use_pyplot=False,
            show_colorbar=True
        )
        axes[row, 1].set_title("LIME")

    fig.suptitle(title, fontsize=14)
    plt.tight_layout()
    plt.show()

**Task 2**

Call `plot_lime_grid` first with the **true positives** and then with the **false negatives**, as you did in Question 2, Task 2.

In [ ]:
# LIME heatmaps for correct predictions (true positives)
plot_lime_grid(
    correct_indices, test_images, test_labels, model, device,
    title="LIME Attributions — Correct Predictions (True Positives)"
)

In [ ]:
# LIME heatmaps for false negatives
plot_lime_grid(
    false_neg_indices, test_images, test_labels, model, device,
    title="LIME Attributions — False Negatives"
)

## Thoughts

Which approach do you think is more informative for the issue at hand?

*Note: Any musings are not marked, they are just to promote thought on the approach*